In [1]:
import pandas as pd
import numpy as np
import re
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# FOR REPRODUCIBILTY
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False 
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [3]:
# USING GPU
# print(torch.cuda.device_count())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train = open("data/train", "r")
lines = train.readlines()

In [5]:
vocab = {}
for i in range(len(lines)):
    if lines[i] == '\n':
        continue
    
    line = lines[i].split(' ')
    if line[1] not in vocab:
        vocab[line[1]] = 1
    else:
        vocab[line[1]] = 1 + vocab[line[1]]    

In [6]:
threshold = 1
unk_dict = {'< unk >': 0}
   
for word in vocab:
    if vocab[word] <= threshold:
        unk_dict['< unk >'] = vocab[word] + unk_dict['< unk >']

In [7]:
final_vocab = vocab.copy()
for word in vocab:
    if final_vocab[word] <= threshold:
        del final_vocab[word]
final_vocab['< unk >'] = unk_dict['< unk >']

In [8]:
words = []
word_index = {}
word_index['pad'] = 0
for word in final_vocab:
    if word not in word_index:
        word_index[word] = len(word_index)

In [9]:
words = []
tag_index = {}
for line in lines:
    if line == '\n':
        continue
    word = line.split(' ')[2].strip()
    if word not in tag_index:
        tag_index[word] = len(tag_index)

tag_index['pad'] = -1

# Bi-LSTM with custom embeddings

In [10]:
def create_pairs(data):
    sent_tag_pair = []
    tx, ty = [], []
    for line in data:
        if line == '\n':
            sent_tag_pair.append([tx, ty])
            tx,ty = [], []
            continue
            
        if line.split(' ')[1] not in word_index:
            tx.append(word_index['< unk >'])
        else:
            tx.append(word_index[line.split(' ')[1]])
        ty.append(tag_index[line.split(' ')[2].strip()])
    sent_tag_pair.append([tx, ty]) 
    return sent_tag_pair

In [11]:
def collate_fn(batch):    
    x, y, x_len = [], [], []
    
    for sent, tag in batch:
        x_len.append(len(sent))
        x.append(torch.LongTensor(sent))
        y.append(torch.LongTensor(tag))
    padding1 = torch.nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=0)
    padding2 = torch.nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=-1)
    
    return padding1, padding2, x_len

In [12]:
batch_size = 8
train_pairs = create_pairs(lines)
train_dataloader = DataLoader(train_pairs, batch_size=batch_size, shuffle=True, pin_memory=True, collate_fn=collate_fn, worker_init_fn=seed_worker, generator=g)

In [13]:
val = open("data/dev", "r")
lines = val.readlines()
val_pair = create_pairs(lines)
val_dataloader = DataLoader(val_pair, batch_size=batch_size, shuffle=False, pin_memory=True, collate_fn=collate_fn, worker_init_fn=seed_worker, generator=g)

In [14]:
num_embed = len(word_index)

In [15]:
class BiLSTM1(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embedding = nn.Embedding(num_embeddings = num_embed, embedding_dim = 100)
        self.bilstm = nn.LSTM(input_size=100, hidden_size=256, num_layers=1, bidirectional=True)
        self.dropout = nn.Dropout(0.33)
        self.linear = nn.Linear(512, 128)
        self.elu = nn.ELU()
        self.classifier1 = nn.Linear(128, 9)
        
    def forward(self, x, x_len):
        x = self.dropout(self.embedding(x))
        x = torch.nn.utils.rnn.pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        x, (hn, cn) = self.bilstm(x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
        x = self.dropout(x)
        x = self.linear(x)
        x = self.elu(x)
        x = self.classifier1(x)
        
        return x

In [16]:
model1 = BiLSTM1().to(device)

In [17]:
model_optim=torch.optim.SGD(model1.parameters(), lr= 0.25, momentum = 0.9)
loss = nn.CrossEntropyLoss(ignore_index=-1).to(device)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(model_optim, mode='max', patience=3, min_lr=0.05, factor = 0.5)

In [18]:
history_train = []
history_val = []
train_dataloader_len = len(train_dataloader)
val_dataloader_len = len(val_dataloader)
highest_val_score = -np.inf
for epoch in range(50):  
    model1.train()
    train_loss = 0.0
    correct_train = 0
    train_accuracy = 0.0
    for i, data in enumerate(train_dataloader, 0):
        final_label = []
        final_output = []
        inputs, label, inputs_len = data
        inputs, label = inputs.to(device), label.to(device)

        model_optim.zero_grad()
        output = model1(inputs, inputs_len)
        output = output.to(device)

        output = output.view(-1, output.size()[-1])
        label = label.view(-1)

        loss1 = loss(output, label)/batch_size
        loss1.backward()
        model_optim.step()

        train_loss += loss1.item()

        _, output = torch.max(output, 1)
        output = output.cpu().detach().numpy()
        label = label.cpu().detach().numpy()
        for ii in range(len(label)):
            if label[ii] != -1:
                final_label.append(label[ii])
                final_output.append(output[ii])
        train_accuracy += f1_score(final_label, final_output, average='macro')
    
    train_loss = train_loss/train_dataloader_len
    train_accuracy = train_accuracy/train_dataloader_len
    history_train.append((train_loss, train_accuracy))
    
    model1.eval()   
    val_loss = 0.0
    val_accuracy = 0.0
    correct_val = 0
    with torch.no_grad():
        for i, data in enumerate(val_dataloader, 0):
            final_label = []
            final_output = []
            inputs, label, inputs_len = data
            inputs, label = inputs.to(device), label.to(device)
            pred = model1(inputs, inputs_len)
            pred = pred.to(device)
            
            pred = pred.view(-1, pred.size()[-1])
            label = label.view(-1)
            
            loss2 = loss(pred, label)/batch_size
            val_loss += loss2.item()
            _, pred = torch.max(pred, 1)
            pred = pred.cpu().detach().numpy()
            label = label.cpu().detach().numpy()
            for ii in range(len(label)):
                if label[ii] != -1:
                    final_label.append(label[ii])
                    final_output.append(pred[ii])
            
            val_accuracy += f1_score(final_label, final_output, average='macro')
            
        val_loss = val_loss/val_dataloader_len
        val_accuracy = val_accuracy/val_dataloader_len
        scheduler.step(val_accuracy)
        history_val.append((val_loss, val_accuracy))
        if val_accuracy > highest_val_score:
            print("Saving model")
            highest_val_score = val_accuracy
            torch.save({'model_state_dict': model1.state_dict()}, 'blstm1.pt')
            
    print("Epoch:", (epoch+1))
    print("Train Loss:", train_loss, "\tTrain F1:", train_accuracy)
    print("Val Loss:", val_loss, "\tVal F1:", val_accuracy)
    print("Current learning rate:", model_optim.param_groups[0]['lr'])
    print("=======================================================================================")

print("Final Val accuracy:", val_accuracy)

Saving model
Epoch: 1
Train Loss: 0.07867799774746598 	Train F1: 0.23591345899195806
Val Loss: 0.06545669195555529 	Val F1: 0.33104536176854715
Current learning rate: 0.25
Saving model
Epoch: 2
Train Loss: 0.05778657596284005 	Train F1: 0.4048153410186125
Val Loss: 0.04726696045412618 	Val F1: 0.49378618162752996
Current learning rate: 0.25
Saving model
Epoch: 3
Train Loss: 0.04772625027449052 	Train F1: 0.5071964385392648
Val Loss: 0.03843432343253771 	Val F1: 0.5912726500275806
Current learning rate: 0.25
Saving model
Epoch: 4
Train Loss: 0.04105193667057126 	Train F1: 0.5666569718309127
Val Loss: 0.03363436125116938 	Val F1: 0.6321070707571649
Current learning rate: 0.25
Saving model
Epoch: 5
Train Loss: 0.036416480243911725 	Train F1: 0.6129901693733626
Val Loss: 0.030330017866906905 	Val F1: 0.672246205902111
Current learning rate: 0.25
Saving model
Epoch: 6
Train Loss: 0.033209020711964625 	Train F1: 0.6463920040702174
Val Loss: 0.027909679907470113 	Val F1: 0.689330620942138
Cur

Epoch: 34
Train Loss: 0.012350828680009163 	Train F1: 0.8518716769188465
Val Loss: 0.019483582818598473 	Val F1: 0.793471594778027
Current learning rate: 0.125
Epoch: 35
Train Loss: 0.012201542736125788 	Train F1: 0.8537382057180424
Val Loss: 0.019251469408431577 	Val F1: 0.7957682153399699
Current learning rate: 0.125
Epoch: 36
Train Loss: 0.012255942605239793 	Train F1: 0.8543555477874506
Val Loss: 0.019256249102582175 	Val F1: 0.7974217758967491
Current learning rate: 0.125
Epoch: 37
Train Loss: 0.011706962330573589 	Train F1: 0.8612756059064756
Val Loss: 0.019230204349384595 	Val F1: 0.7955498542667698
Current learning rate: 0.0625
Saving model
Epoch: 38
Train Loss: 0.011689812270011266 	Train F1: 0.8598685525939438
Val Loss: 0.018976059118817997 	Val F1: 0.7993409513360921
Current learning rate: 0.0625
Saving model
Epoch: 39
Train Loss: 0.011355801283452635 	Train F1: 0.866671281526383
Val Loss: 0.019295098170367356 	Val F1: 0.8018556544287061
Current learning rate: 0.0625
Epoch: 

In [19]:
final_output = []
final_label = []
model1.load_state_dict(torch.load('blstm1.pt'), strict=False)
model1.eval()
with torch.no_grad():
    for i, data in enumerate(val_dataloader, 0):

        inputs, label, inputs_len = data
        inputs, label = inputs.to(device), label.to(device)
        pred = model1(inputs, inputs_len)
        pred = pred.to(device)

        pred = pred.view(-1, pred.size()[-1])
        label = label.view(-1)
        label = label.cpu().detach().numpy()

        _, pred = torch.max(pred, 1)
        pred = pred.cpu().detach().numpy()
        
        for ii in range(len(label)):
            if label[ii] != -1:
                final_output.append(pred[ii])
                final_label.append(label[ii])

In [20]:
val = open('data/dev', 'r')
lines = val.readlines()

all_sentences = []
cnt = 0
for line in lines:
    if line == '\n':
         all_sentences.append(line)
    else:
        cnt += 1
        all_sentences.append(line.split(' '))
        

In [21]:
i = 0
val_key = {val:key for key, val in tag_index.items()}
with open("eval_dev1.out", "w") as f:
    for splits in all_sentences:
            if splits == "\n":
                f.write("\n")
            else:

                idx, word, tag = splits
                f.write(f"{idx} {word} {tag.strip()} {val_key[final_output[i]]}\n")
                i += 1
i = 0         
with open("dev1.out", "w") as f:
    for splits in all_sentences:
            if splits == "\n":
                f.write("\n")
            else:

                idx, word, tag = splits
                f.write(f"{idx} {word} {val_key[final_output[i]]}\n")
                i += 1

In [22]:
def create_pairs_test(data):
    sent_tag_pair = []
    tx = []
    for line in data:
        if line == '\n':
            sent_tag_pair.append(tx)
            tx = []
            continue
            
        if line.split(' ')[1] not in word_index:
            tx.append(word_index['< unk >'])
        else:
            tx.append(word_index[line.split(' ')[1]])
    sent_tag_pair.append(tx) 
    return sent_tag_pair

In [23]:
def collate_fn_test(batch):    
    x, x_len = [], []
    
    for sent in batch:
        x_len.append(len(sent))
        x.append(torch.LongTensor(sent))
    padding1 = torch.nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=0)
    
    return padding1, x_len

In [24]:
test = open("data/test", "r")
lines = test.readlines()
test_pair = create_pairs_test(lines)
test_dataloader = DataLoader(test_pair, batch_size=8, shuffle=False, pin_memory=True, collate_fn=collate_fn_test, worker_init_fn=seed_worker, generator=g)

In [25]:
final_output = []
final_label = []
model1.load_state_dict(torch.load('blstm1.pt'), strict=False)
model1.eval()
with torch.no_grad():
    for i, data in enumerate(test_dataloader, 0):

        inputs, inputs_len = data
        inputs = inputs.to(device)
        pred = model1(inputs, inputs_len)
        pred = pred.to(device)

        pred = pred.view(-1, pred.size()[-1])

        _, pred = torch.max(pred, 1)
        pred = pred.cpu().detach().numpy()
        
        for ii in range(len(pred)):
            if pred[ii] != -1:
                final_output.append(pred[ii])

test = open('data/test', 'r')
lines = test.readlines()

all_sentences = []
cnt = 0
for line in lines:
    if line == '\n':
         all_sentences.append(line)
    else:
        cnt += 1
        all_sentences.append(line.split(' '))
        
i = 0         
test_key = {test:key for key, test in tag_index.items()}
with open("test1.out", "w") as f:
    for splits in all_sentences:
            if splits == "\n":
                f.write("\n")
            else:

                idx, word = splits
                f.write(f"{idx} {word.strip()} {test_key[final_output[i]]}\n")
                i += 1


# BiLSTM with GloVe embeddings

In [27]:
glove_embeddings = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        embedding = np.array([float(val) for val in values[1:]])
        glove_embeddings[word] = embedding

In [28]:
# embeddings = np.zeros((len(word_index), 100))
# for word, index in word_index.items():
#     if word.lower() in glove_embeddings:
#         embeddings[index] = glove_embeddings[word.lower()]
#     elif word in glove_embeddings:
#         embeddings[index] = glove_embeddings[word]
#     else:
#         embeddings[index] = np.random.uniform(low=-0.25, high=0.25, size=100)

In [29]:
embeddings = np.zeros((len(word_index), 101))
temp_num=np.sqrt(3.0/100.0)
temp = np.zeros((len(word_index), 100))
for word, index in word_index.items():
    if (word.lower() == word) and (word in glove_embeddings):
        temp[index] = glove_embeddings[word.lower()]
        embeddings[index] = np.append(temp[index], 0)
    elif (word.lower() == word) and (word not in glove_embeddings):
        temp[index] = np.random.uniform(low=-temp_num, high=temp_num, size=100)
        embeddings[index] = np.append(temp[index], 2)
    elif (word.lower() != word) and (word in glove_embeddings):
        temp[index] = glove_embeddings[word]
        embeddings[index] = np.append(temp[index], 1)
    else:
        temp[index] = np.random.uniform(low=-temp_num, high=temp_num, size=100)
        embeddings[index] = np.append(temp[index], 2)

In [30]:
embedding_layer = nn.Embedding(len(word_index), 101)
embedding_layer.load_state_dict({"weight":torch.FloatTensor(embeddings)})

<All keys matched successfully>

In [31]:
def initialize_linear_layer(layer):
    nn.init.xavier_normal_(layer.weight.data)
    nn.init.normal_(layer.bias.data)

def initialize_lstm_layer(layer):
    for param in layer.parameters():
        if len(param.shape) >= 2:
            nn.init.orthogonal_(param.data)
        else:
            nn.init.normal_(param.data)

class BiLSTM2(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embedding = embedding_layer
        self.bilstm = nn.LSTM(input_size=101, hidden_size=256, num_layers=1, bidirectional=True)
        initialize_lstm_layer(self.bilstm)
        self.dropout = nn.Dropout(0.33)
        self.linear = nn.Linear(512, 128)
        initialize_linear_layer(self.linear)
        self.elu = nn.ELU()
        self.classifier1 = nn.Linear(128, 9)
        initialize_linear_layer(self.classifier1)
    
        
    def forward(self, x, x_len):
        x = self.dropout(self.embedding(x))
        x = torch.nn.utils.rnn.pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        x, (hn, cn) = self.bilstm(x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
        x = self.dropout(x)
        x = self.linear(x)
        x = self.elu(x)
#         x = self.classifier1(x)
        x = self.classifier1(x)
        
        return x

In [32]:
model2 = BiLSTM2().to(device)

In [33]:
model_optim=torch.optim.SGD(model2.parameters(), lr= 0.25, momentum = 0.95)
loss = nn.CrossEntropyLoss(ignore_index=-1).to(device)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(model_optim, mode='max', patience=2, min_lr=0.05, factor = 0.5)

In [34]:
history_train = []
history_val = []
train_dataloader_len = len(train_dataloader)
val_dataloader_len = len(val_dataloader)
highest_val_score = -np.inf
for epoch in range(55):  
    model2.train()
    train_loss = 0.0
    correct_train = 0
    train_accuracy = 0.0
    for i, data in enumerate(train_dataloader, 0):
        final_label = []
        final_output = []
        inputs, label, inputs_len = data
        inputs, label = inputs.to(device), label.to(device)

        model_optim.zero_grad()
        output = model2(inputs, inputs_len)
        output = output.to(device)

        output = output.view(-1, output.size()[-1])
        label = label.view(-1)

        loss1 = loss(output, label)/batch_size
        loss1.backward()
        model_optim.step()

        train_loss += loss1.item()

        _, output = torch.max(output, 1)
        output = output.cpu().detach().numpy()
        label = label.cpu().detach().numpy()
        for ii in range(len(label)):
            if label[ii] != -1:
                final_label.append(label[ii])
                final_output.append(output[ii])
        train_accuracy += f1_score(final_label, final_output, average='macro')
    
    train_loss = train_loss/train_dataloader_len
    train_accuracy = train_accuracy/train_dataloader_len
    history_train.append((train_loss, train_accuracy))
    
    model2.eval()   
    val_loss = 0.0
    val_accuracy = 0.0
    correct_val = 0
    with torch.no_grad():
        for i, data in enumerate(val_dataloader, 0):
            final_label = []
            final_output = []
            inputs, label, inputs_len = data
            inputs, label = inputs.to(device), label.to(device)
            pred = model2(inputs, inputs_len)
            pred = pred.to(device)
            
            pred = pred.view(-1, pred.size()[-1])
            label = label.view(-1)
            
            loss2 = loss(pred, label)/batch_size
            val_loss += loss2.item()
            _, pred = torch.max(pred, 1)
            pred = pred.cpu().detach().numpy()
            label = label.cpu().detach().numpy()
            for ii in range(len(label)):
                if label[ii] != -1:
                    final_label.append(label[ii])
                    final_output.append(pred[ii])
            val_accuracy += f1_score(final_label, final_output, average='macro')
            
        val_loss = val_loss/val_dataloader_len
        val_accuracy = val_accuracy/val_dataloader_len
        scheduler.step(val_accuracy)
        history_val.append((val_loss, val_accuracy))
        if val_accuracy > highest_val_score:
            print("Saving model")
            highest_val_score = val_accuracy
            torch.save({'model_state_dict': model2.state_dict()}, 'blstm2.pt')
            
    print("Epoch:", (epoch+1))
    print("Train Loss:", train_loss, "\tTrain F1:", train_accuracy)
    print("Val Loss:", val_loss, "\tVal F1:", val_accuracy)
    print("Current learning rate:", model_optim.param_groups[0]['lr'])
    print("=======================================================================================")

print("Final Val accuracy:", val_accuracy)

Saving model
Epoch: 1
Train Loss: 0.0471497024896504 	Train F1: 0.39625359162286616
Val Loss: 0.03453427309013178 	Val F1: 0.5490869725306171
Current learning rate: 0.25
Saving model
Epoch: 2
Train Loss: 0.02731319518015385 	Train F1: 0.6160794072332325
Val Loss: 0.02502042608726437 	Val F1: 0.6628649676814182
Current learning rate: 0.25
Saving model
Epoch: 3
Train Loss: 0.01928349273287261 	Train F1: 0.7356017806285772
Val Loss: 0.024047875464791948 	Val F1: 0.7002817586077482
Current learning rate: 0.25
Saving model
Epoch: 4
Train Loss: 0.014979772003473895 	Train F1: 0.8004019640633508
Val Loss: 0.01880057969783635 	Val F1: 0.7493376617010018
Current learning rate: 0.25
Saving model
Epoch: 5
Train Loss: 0.012269328069052874 	Train F1: 0.8358121978634601
Val Loss: 0.01845233304767692 	Val F1: 0.791353176618177
Current learning rate: 0.25
Epoch: 6
Train Loss: 0.010618202063298995 	Train F1: 0.8654851635330121
Val Loss: 0.01880726633348807 	Val F1: 0.7844529786749549
Current learning r

Epoch: 34
Train Loss: 0.002425316671603574 	Train F1: 0.966929665427905
Val Loss: 0.015834246621577282 	Val F1: 0.8476351232560113
Current learning rate: 0.05
Epoch: 35
Train Loss: 0.002379694611325289 	Train F1: 0.9697128128118093
Val Loss: 0.016093871547112594 	Val F1: 0.8465794899477006
Current learning rate: 0.05
Epoch: 36
Train Loss: 0.0023959812382354346 	Train F1: 0.9699514691130172
Val Loss: 0.01608705014845386 	Val F1: 0.8478485074418692
Current learning rate: 0.05
Epoch: 37
Train Loss: 0.0023644579484196826 	Train F1: 0.9692606205657636
Val Loss: 0.015794912256052836 	Val F1: 0.8476792285573318
Current learning rate: 0.05
Epoch: 38
Train Loss: 0.0023388570395715628 	Train F1: 0.9694453844420423
Val Loss: 0.016168860858158815 	Val F1: 0.8463237665907798
Current learning rate: 0.05
Saving model
Epoch: 39
Train Loss: 0.0022933447988557863 	Train F1: 0.9709804811613686
Val Loss: 0.016210232250147727 	Val F1: 0.8540286842349515
Current learning rate: 0.05
Epoch: 40
Train Loss: 0.0

In [35]:
final_output = []
final_label = []
model2.load_state_dict(torch.load('blstm2.pt'), strict=False)
model2.eval()
with torch.no_grad():
    for i, data in enumerate(val_dataloader, 0):

        inputs, label, inputs_len = data
        inputs, label = inputs.to(device), label.to(device)
        pred = model2(inputs, inputs_len)
        pred = pred.to(device)

        pred = pred.view(-1, pred.size()[-1])
        label = label.view(-1)
        label = label.cpu().detach().numpy()

        _, pred = torch.max(pred, 1)
        pred = pred.cpu().detach().numpy()
        
        for ii in range(len(label)):
            if label[ii] != -1:
                final_output.append(pred[ii])
                final_label.append(label[ii])

In [36]:
val = open('data/dev', 'r')
lines = val.readlines()

all_sentences = []
cnt = 0
for line in lines:
    if line == '\n':
         all_sentences.append(line)
    else:
        cnt += 1
        all_sentences.append(line.split(' '))

In [37]:
i = 0
val_key = {val:key for key, val in tag_index.items()}
with open("eval_dev2.out", "w") as f:
    for splits in all_sentences:
            if splits == "\n":
                f.write("\n")
            else:

                idx, word, tag = splits
                f.write(f"{idx} {word} {tag.strip()} {val_key[final_output[i]]}\n")
                i += 1
i = 0         
with open("dev2.out", "w") as f:
    for splits in all_sentences:
            if splits == "\n":
                f.write("\n")
            else:

                idx, word, tag = splits
                f.write(f"{idx} {word} {val_key[final_output[i]]}\n")
                i += 1

In [39]:
final_output = []
final_label = []
model2.load_state_dict(torch.load('blstm2.pt'), strict=False)
model2.eval()
with torch.no_grad():
    for i, data in enumerate(test_dataloader, 0):

        inputs, inputs_len = data
        inputs = inputs.to(device)
        pred = model2(inputs, inputs_len)
        pred = pred.to(device)

        pred = pred.view(-1, pred.size()[-1])

        _, pred = torch.max(pred, 1)
        pred = pred.cpu().detach().numpy()
        
        for ii in range(len(pred)):
            if pred[ii] != -1:
                final_output.append(pred[ii])

test = open('data/test', 'r')
lines = test.readlines()

all_sentences = []
cnt = 0
for line in lines:
    if line == '\n':
         all_sentences.append(line)
    else:
        cnt += 1
        all_sentences.append(line.split(' '))
        
i = 0         
test_key = {test:key for key, test in tag_index.items()}
with open("test2.out", "w") as f:
    for splits in all_sentences:
            if splits == "\n":
                f.write("\n")
            else:

                idx, word = splits
                f.write(f"{idx} {word.strip()} {test_key[final_output[i]]}\n")
                i += 1
